In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, find_peaks
from scipy.ndimage import gaussian_filter1d
from sklearn.preprocessing import scale
import seaborn as sns
from tqdm import tqdm

In [ ]:
#Acquisition of data
file_data = {"Filename":[], "DataType":[], "Condition":[], "Context":[], "Timepoint":[],  "Subject":[]}
folder = "collected_data"
files = os.listdir(folder)

for file in files:
    # Need to filter and assign each file to a data type (e.g., "photometry", "dlc", "video_timestamps").
	# Need to extract metadata, including group name, context, test time point, and subject ID (e.g., "con", "context A", "recent").
    file_data["Filename"].append(os.path.join(folder, file))
    file_data["DataType"].append(data_type)
    file_data["Condition"].append(condition)
    file_data["Context"].append(context)
    file_data["Timepoint"].append(timepoint)
    file_data["Subject"].append(subject)

file_df = pd.DataFrame(file_data)
pd.set_option('display.max_rows', None)
file_df

In [ ]:
# Peak extraction
def filter_channel_data(pd_data, led_state, region="Region0G", sigma=1.):
    led_state = str(led_state)
    channel = pd_data.query("LedState=={}".format(led_state))[region].values
    ch_ts = pd_data.query("LedState=={}".format(led_state))["Timestamp"].values
    f_ch = interp1d(ch_ts, channel, fill_value="extrapolate")
    ch_interp = f_ch(ts)
    return gaussian_filter1d(ch_interp, sigma)

for context in ["homecage", "A", "B"]:
    session_files = file_df.query("Condition == @condition and Context == @context and Timepoint == @timepoint and Subject == @subject")

    photometry_file = session_files.query("DataType == 'photometry'")["Filename"].values[0]
    data = pd.read_csv(photometry_file)
    ts = np.arange(data["Timestamp"][0], data["Timestamp"].values[-1], 1/fs)
    
    # filter the original photometry data in each channel
    ch1_interp = filter_channel_data(data, led_state=1)
    ch2_interp = filter_channel_data(data, led_state=2)
        
    # apply the Butterworth high-pass filter to the normalized data within a given time window
    ts, ch1_interp, ch2_interp = ts[start*fs:end*fs], ch1_interp[start*fs:end*fs], ch2_interp[start*fs:end*fs]
    ts_rel = ts - ts[0]
    b, a = butter(2, 0.05, btype="high", fs=10)
    norm = filtfilt(b, a, (ch2_interp - ch1_interp) / ch1_interp)

    # peak extraction
    mad = np.median(np.abs(norm - np.median(norm)))
    thresh1 = np.median(norm) + 2*mad

    point_under_thresh = norm[norm < thresh1]
    mad2 = np.median(np.abs(point_under_thresh - np.median(point_under_thresh)))
    thresh2 = np.median(point_under_thresh) + 2*mad2

    norm_smooth = gaussian_filter1d(norm, 10)
    peaks, peak_data = find_peaks(norm_smooth, height=thresh2)
    
    #Peak count
    peak_count = len(peaks)
    print("Number of peaks:", peak_count)
    print(2*mad2)